In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('../input/health-insurance-cross-sell-prediction/train.csv')
print(data.shape)
data.head()

# from sklearn.utils import shuffle
# data = shuffle(data)
# train = data[:20000]
# val = data[20000:30000]
# print(train.shape)
# print(val.shape)
# train.to_csv('train.csv')
# val.to_csv('val.csv')

In [ ]:
data[['Vehicle_Damage', 'Response']].groupby('Vehicle_Damage').mean()

In [ ]:
def preprocessing(data):
    trans_data = data.copy()
    idx = trans_data.pop('id')
#y = data.pop('Response')
    trans_data = trans_data.join(pd.get_dummies(data['Gender']))
    trans_data = trans_data.join(pd.get_dummies(data['Vehicle_Damage']))
    trans_data = trans_data.join(pd.get_dummies(data['Vehicle_Age']))
    trans_data.pop('Gender')
    trans_data.pop('Policy_Sales_Channel')
    trans_data.pop('Vehicle_Damage')
    trans_data.pop('Vehicle_Age')

    trans_data[['Age', 'Annual_Premium']]  = (data[['Age', 'Annual_Premium']]-data[['Age', 'Annual_Premium']].mean())/data[['Age', 'Annual_Premium']].std()
    y = trans_data.pop('Response')
    return idx, trans_data, y

In [ ]:
idx, data, y = preprocessing(data)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(roc_auc_score(y_test, y_pred))


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression

classifiers = [
    KNeighborsClassifier(3),
   # SVC(probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
   # QuadraticDiscriminantAnalysis(),
   # LogisticRegression()
]

log_cols = ["Classifier", "Accuracy"]
log = pd.DataFrame(columns=log_cols)

sss = StratifiedShuffleSplit(n_splits=10, test_size=0.1, random_state=0)

acc_dict = {}


for clf in classifiers:
    name = clf.__class__.__name__
    clf.fit(X_train, y_train)
    train_predictions = clf.predict(X_test)
    acc = roc_auc_score(y_test, train_predictions)
    print(name)
    if name in acc_dict:
        acc_dict[name] += acc
    else:
        acc_dict[name] = acc

for clf in acc_dict:
    acc_dict[clf] = acc_dict[clf] / 10.0
    log_entry = pd.DataFrame([[clf, acc_dict[clf]]], columns = log_cols)
    log = log.append(log_entry)

plt.xlabel('Accuracy')
plt.title('Classifier Accuracy')

sns.set_color_codes("muted")
sns.barplot(x = 'Accuracy', y = 'Classifier', data = log, color = "b")
log

In [ ]:
clf = GaussianNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(roc_auc_score(y_test, y_pred))

ss = pd.read_csv('../input/health-insurance-cross-sell-prediction/sample_submission.csv')
ss

In [ ]:
test = pd.read_csv('../input/health-insurance-cross-sell-prediction/test.csv')
#data.describe()

idx = test.pop('id')
#y = data.pop('Response')
test = test.join(pd.get_dummies(test['Gender']))
test = test.join(pd.get_dummies(test['Vehicle_Damage']))
test = test.join(pd.get_dummies(test['Vehicle_Age']))
test.pop('Gender')
test.pop('Policy_Sales_Channel')
test.pop('Vehicle_Damage')
test.pop('Vehicle_Age')

test[['Age', 'Annual_Premium']]  = (data[['Age', 'Annual_Premium']]-data[['Age', 'Annual_Premium']].mean())/data[['Age', 'Annual_Premium']].std()
ans = clf.predict(test)
ans = pd.DataFrame(data = {'id': idx,'Response': ans})
ans
ans.to_csv('prediction.csv')